<a href="https://colab.research.google.com/github/saishrane06/Feature-Selection-Techniques/blob/main/Chi2_Statistical_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download josemauricioneuro/lung-cancer-patients-mrna-microarray

 99% 124M/125M [00:02<00:00, 57.7MB/s] 
100% 125M/125M [00:02<00:00, 45.3MB/s]


In [2]:
! unzip lung-cancer-patients-mrna-microarray.zip

Archive:  lung-cancer-patients-mrna-microarray.zip
  inflating: clean/complete_dataframe.csv  
  inflating: raw/DCLungStudy_Clinical_Covariates_with_Hgrade.xls  
  inflating: raw/DCLungStudy_Not_In_Study_Reasons.xls  
  inflating: raw/DCLungStudy_dChip-Processed_microarray_data.xls  
  inflating: raw/DClungStudy_Clinical_Covariate_Definitions.xls  


In [3]:
import pandas as pd
import numpy as np

In [4]:
lc_df = pd.read_csv("clean/complete_dataframe.csv")
lc_df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (22309) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,...,SURGICAL_MARGINS,PATHOLOGIC_N_STAGE,PATHOLOGIC_T_STAGE,MEDIAN_INTENSITY_UNNORMALIZED,PCT_ARRAY_OUTLIER,PCT_SINGLE_OUTLIER,WARNING,LABORATORY_BATCH,Histologic grade,High_risk
0,NCI_U133A_93L,10.937022,6.105887,6.657732,8.543040,4.088473,8.208775,5.248811,5.053609,7.042420,...,ALL MARGINS PATHOLOGICALLY NEGATIVE,"N0, ACCORDING TO AJCC CRITERIA","T2, ACCORDING TO AJCC CRITERIA",193.0,2.643,0.148,NaN,NaN,Moderate Differentiation,1.0
1,NCI_U133A_70L,9.667748,6.203607,7.444259,8.800833,4.460766,7.960886,5.133321,4.940906,7.618997,...,ALL MARGINS PATHOLOGICALLY NEGATIVE,"N1, ACCORDING TO AJCC CRITERIA","T2, ACCORDING TO AJCC CRITERIA",192.0,1.450,0.184,NaN,NaN,POORLY DIFFERENTIATED,0.0
2,NCI_U133A_87L,10.083438,6.416165,7.145344,8.625512,4.135750,8.499000,5.134877,4.836854,7.089821,...,ALL MARGINS PATHOLOGICALLY NEGATIVE,"N0, ACCORDING TO AJCC CRITERIA","T1, ACCORDING TO AJCC CRITERIA",192.0,1.634,0.142,NaN,NaN,WELL DIFFERENTIATED,0.0
3,NCI_U133A_64L,10.394160,6.307711,7.290440,8.761087,4.112526,8.463786,5.476750,4.998902,7.725870,...,ALL MARGINS PATHOLOGICALLY NEGATIVE,"N0, ACCORDING TO AJCC CRITERIA","T2, ACCORDING TO AJCC CRITERIA",192.0,2.850,0.350,NaN,NaN,POORLY DIFFERENTIATED,0.0
4,NCI_U133A_99L,10.890421,6.307370,6.835464,8.524622,4.200539,7.584603,5.352621,4.941038,5.878343,...,ALL MARGINS PATHOLOGICALLY NEGATIVE,"N0, ACCORDING TO AJCC CRITERIA","T2, ACCORDING TO AJCC CRITERIA",193.0,3.666,0.244,NaN,NaN,POORLY DIFFERENTIATED,0.0


In [5]:
lc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Columns: 22313 entries, Unnamed: 0 to High_risk
dtypes: float64(22293), int64(3), object(17)
memory usage: 75.2+ MB


In [6]:
for i in lc_df.columns:
  if lc_df[i].dtypes == 'O':
    print(i)
    print("\n")
    print(lc_df[i].unique())
    print("\n")

Unnamed: 0


['NCI_U133A_93L' 'NCI_U133A_70L' 'NCI_U133A_87L' 'NCI_U133A_64L'
 'NCI_U133A_99L' 'CL20041119183AA' 'NCI_Lung270_U133A' 'Moff1864H'
 'NCI_Lung75_U133A' 'NCI_Lung23_U133A' 'Moff0928E' 'NCI_Lung91_U133A'
 'CL2005060333AA' 'NCI_U133A_23L' 'NCI_U133A_25L' 'NCI_U133A_26L'
 'NCI_U133A_38L' 'NCI_U133A_68L' 'NCI_U133A_32L' 'NCI_U133A_31L'
 'NCI_Lung44_U133A' 'NCI_U133A_10L' 'NCI_Lung99_U133A' 'CL20041119184AA'
 'NCI_Lung247_U133A' 'NCI_Lung203_U133A' 'NCI_Lung19_U133A'
 'NCI_Lung208_U133A' 'Moff1487F' 'CL2005060334AA' 'NCI_Lung259_U133A'
 'Moff3730B' 'Moff2603I' 'Moff2333J' 'Moff2690H' 'NCI_Lung14_U133A'
 'Moff3401F' 'NCI_Lung65_U133A' 'NCI_U133A_46L' 'NCI_U133A_9L' 'Moff3053B'
 'Moff0978B' 'Moff4112G' 'NCI_U133A_52L' 'Moff0711B' 'Moff1835I'
 'CL2004111045AA' 'NCI_Lung80_U133A' 'Moff1576F' 'Moff2955B'
 'NCI_Lung281_U133' 'CL2005060344AA' 'NCI_Lung296_U133' 'NCI_Lung46_U133A'
 'NCI_U133A_8L' 'NCI_Lung212_U133A' 'NCI_Lung67_U133A' 'NCI_Lung16_U133A'
 'NCI_Lung85_U133A' 'NCI_Lung20_U

In [7]:
lc_df.isna().any()

Unnamed: 0            False
1007_s_at             False
1053_at               False
117_at                False
121_at                False
                      ...  
PCT_SINGLE_OUTLIER    False
WARNING                True
LABORATORY_BATCH       True
Histologic grade       True
High_risk             False
Length: 22313, dtype: bool

In [8]:
## To find the count of null values
for i in lc_df.columns:
  if lc_df[i].isna().any():
    print(i)
    print(lc_df[i].isna().sum())

FIRST_PROGRESSION_OR_RELAPSE
70
MONTHS_TO_FIRST_PROGRESSION
264
MTHS_TO_LAST_CLINICAL_ASSESSMENT
10
SMOKING_HISTORY
2
SURGICAL_MARGINS
2
PATHOLOGIC_N_STAGE
2
PATHOLOGIC_T_STAGE
2
WARNING
410
LABORATORY_BATCH
104
Histologic grade
7


In [9]:
lc_df.shape

(442, 22313)

In [10]:
## Since more than fifty percent of the data are nan values we will delete/drop MONTHS_TO_FIRST_PROGRESSION column
lc_df = lc_df.drop("MONTHS_TO_FIRST_PROGRESSION",axis=1)

In [11]:
lc_df = lc_df[lc_df['MTHS_TO_LAST_CLINICAL_ASSESSMENT'].notna()]
lc_df = lc_df[lc_df['SURGICAL_MARGINS'].notna()]
lc_df = lc_df[lc_df['PATHOLOGIC_N_STAGE'].notna()]
lc_df = lc_df[lc_df['PATHOLOGIC_T_STAGE'].notna()]
lc_df = lc_df[lc_df['Histologic grade'].notna()]

In [12]:
## beacuse this column dtype is object we will fill nan values after labelencoding
lc_df['FIRST_PROGRESSION_OR_RELAPSE'].unique()

array(['No', 'Yes', nan, 'Unknown'], dtype=object)

In [13]:
lc_df['LABORATORY_BATCH'] = lc_df['LABORATORY_BATCH'].fillna(lc_df['LABORATORY_BATCH'].mean())

In [14]:
from sklearn import preprocessing
label_encoding = preprocessing.LabelEncoder()
lc_df['BlindedIDs'] = label_encoding.fit_transform(lc_df['BlindedIDs'])
lc_df['DC_STUDY_ID'] = label_encoding.fit_transform(lc_df['DC_STUDY_ID'])
lc_df['SITE'] = label_encoding.fit_transform(lc_df['SITE'])
lc_df['TESTTYPE'] = label_encoding.fit_transform(lc_df['TESTTYPE'])
lc_df['GENDER'] = label_encoding.fit_transform(lc_df['GENDER'])
lc_df['RACE'] = label_encoding.fit_transform(lc_df['RACE'])
lc_df['ADJUVANT_CHEMO'] = label_encoding.fit_transform(lc_df['ADJUVANT_CHEMO'])
lc_df['ADJUVANT_RT'] = label_encoding.fit_transform(lc_df['ADJUVANT_RT'])
lc_df['VITAL_STATUS'] = label_encoding.fit_transform(lc_df['VITAL_STATUS'])
lc_df['FIRST_PROGRESSION_OR_RELAPSE'] = label_encoding.fit_transform(lc_df['FIRST_PROGRESSION_OR_RELAPSE'])
lc_df['SMOKING_HISTORY'] = label_encoding.fit_transform(lc_df['SMOKING_HISTORY'])
lc_df['SURGICAL_MARGINS'] = label_encoding.fit_transform(lc_df['SURGICAL_MARGINS'])
lc_df['PATHOLOGIC_N_STAGE'] = label_encoding.fit_transform(lc_df['PATHOLOGIC_N_STAGE'])
lc_df['PATHOLOGIC_T_STAGE'] = label_encoding.fit_transform(lc_df['PATHOLOGIC_T_STAGE'])
lc_df['WARNING'] = label_encoding.fit_transform(lc_df['WARNING'])
lc_df['Histologic grade'] = label_encoding.fit_transform(lc_df['Histologic grade'])

In [15]:
lc_df['FIRST_PROGRESSION_OR_RELAPSE'] = lc_df['FIRST_PROGRESSION_OR_RELAPSE'].fillna(lc_df['FIRST_PROGRESSION_OR_RELAPSE'].mean())

In [16]:
lc_df = lc_df.drop("Unnamed: 0",axis=1)

In [17]:
lc_df.head()

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,SURGICAL_MARGINS,PATHOLOGIC_N_STAGE,PATHOLOGIC_T_STAGE,MEDIAN_INTENSITY_UNNORMALIZED,PCT_ARRAY_OUTLIER,PCT_SINGLE_OUTLIER,WARNING,LABORATORY_BATCH,Histologic grade,High_risk
0,10.937022,6.105887,6.657732,8.543040,4.088473,8.208775,5.248811,5.053609,7.042420,4.113564,...,0,0,1,193.0,2.643,0.148,1,13.009231,0,1.0
1,9.667748,6.203607,7.444259,8.800833,4.460766,7.960886,5.133321,4.940906,7.618997,4.114407,...,0,1,1,192.0,1.450,0.184,1,13.009231,1,0.0
2,10.083438,6.416165,7.145344,8.625512,4.135750,8.499000,5.134877,4.836854,7.089821,4.171350,...,0,0,0,192.0,1.634,0.142,1,13.009231,2,0.0
3,10.394160,6.307711,7.290440,8.761087,4.112526,8.463786,5.476750,4.998902,7.725870,4.183584,...,0,0,1,192.0,2.850,0.350,1,13.009231,1,0.0
4,10.890421,6.307370,6.835464,8.524622,4.200539,7.584603,5.352621,4.941038,5.878343,4.005468,...,0,0,1,193.0,3.666,0.244,1,13.009231,1,0.0


In [18]:
X = lc_df.drop("High_risk",axis=1)
y = lc_df['High_risk']

In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)
X_train.shape,X_test.shape

((297, 22310), (128, 22310))

In [20]:
from sklearn.feature_selection import chi2
f_p_values = chi2(X_train,y_train)

In [21]:
f_p_values

(array([3.26464191e-02, 1.16618270e-01, 1.45997617e-05, ...,
        5.52105009e-03, 1.00791729e+00, 1.05351596e-01]),
 array([0.85661625, 0.73273159, 0.99695132, ..., 0.94076868, 0.31540231,
        0.74549964]))

In [22]:
import pandas as pd
p_values=pd.Series(f_p_values[1])
p_values.index=X_train.columns
p_values

1007_s_at             0.856616
1053_at               0.732732
117_at                0.996951
121_at                0.955064
1255_g_at             0.757135
                        ...   
PCT_ARRAY_OUTLIER     0.280157
PCT_SINGLE_OUTLIER    0.656321
WARNING               0.940769
LABORATORY_BATCH      0.315402
Histologic grade      0.745500
Length: 22310, dtype: float64

In [23]:
p_values.sort_index(ascending=False)

target          1.072575e-41
WARNING         9.407687e-01
VITAL_STATUS    7.424445e-09
TESTTYPE        6.586391e-01
Stratagene               NaN
                    ...     
1255_g_at       7.571350e-01
121_at          9.550642e-01
117_at          9.969513e-01
1053_at         7.327316e-01
1007_s_at       8.566162e-01
Length: 22310, dtype: float64

### **Observation**

Target, WARNING, VITAL_STATUS, TESTTYPE are the top 4 important columns in this data set.